# Module Feedback

This system records the responses of students on their learning experience at university.

![rel](https://sqlzoo.net/w/images/7/71/Module-feedback.png)

In [1]:
import getpass
import psycopg2
from sqlalchemy import create_engine
import pandas as pd
import numpy as np
pwd = getpass.getpass()
engine = create_engine(
    'postgresql+psycopg2://postgres:%s@localhost/sqlzoo' % (pwd))
pd.set_option('display.max_rows', 100)

 ····


## 1. Find the student name from a matriculation number

**Find the name of the student with number 50200100**

In [2]:
insspr = pd.concat(list(pd.read_sql_table('INS_SPR', engine, chunksize=1000)))
insmod = pd.concat(list(pd.read_sql_table('INS_MOD', engine, chunksize=1000)))
insprs = pd.concat(list(pd.read_sql_table('INS_PRS', engine, chunksize=1000)))
camsmo = pd.concat(list(pd.read_sql_table('CAM_SMO', engine, chunksize=1000)))
insres = pd.concat(list(pd.read_sql_table('INS_RES', engine, chunksize=1000)))
insque = pd.concat(list(pd.read_sql_table('INS_QUE', engine, chunksize=1000)))
inscat = pd.concat(list(pd.read_sql_table('INS_CAT', engine, chunksize=1000)))

# AYR_CODE length 7, so trim it
camsmo['AYR_CODE'] = camsmo['AYR_CODE'].str.strip()
insres['AYR_CODE'] = insres['AYR_CODE'].str.strip()

In [3]:
insspr.loc[insspr['SPR_CODE']=='50200100', ['SPR_FNM1', 'SPR_SURN']]

,SPR_FNM1,SPR_SURN
2,Tom,Cotton


## 2. Find the modules studied by a student

**Show the module code and module name for modules studied by the student with number 50200100 in session 2016/7 TR1**

In [4]:
(insmod.merge(camsmo, on='MOD_CODE')
 .query('(`SPR_CODE`=="50200100") & (`AYR_CODE`=="2016/7") & (`PSL_CODE`=="TR1")')
 [['MOD_CODE', 'MOD_NAME']])

,MOD_CODE,MOD_NAME
2,CSN08101,Systems and Services
132,INF08104,Database Systems
246,SET08108,Software Development 2


## 3. Find the modules and module leader studied by a student

**Show the module code and module name and details of the module leader for modules studied by the student with number 50200100 in session 2016/7 TR1**

In [5]:
(camsmo.merge(insmod, on='MOD_CODE')
 .merge(insprs, on="PRS_CODE")
 .query('(SPR_CODE=="50200100") & (AYR_CODE=="2016/7") & (PSL_CODE=="TR1")')
 [['MOD_CODE', 'MOD_NAME', 'PRS_CODE', 'PRS_FNM1', 'PRS_SURN']])

,MOD_CODE,MOD_NAME,PRS_CODE,PRS_FNM1,PRS_SURN
2,CSN08101,Systems and Services,40000008,James,Jackson
97,INF08104,Database Systems,40000036,Andrew,Cumming
246,SET08108,Software Development 2,40000408,Neil,Urquhart


## 4. Show the scores for module SET08108

**Show the Percentage of students who gave 4 or 5 to module SET08108 in session 2016/7 TR1**

(note that this is not real data, these responses were randomly generated)

In [6]:
a = (insres
 .assign(score=np.floor(insres['RES_VALU']/4))
 .query('(MOD_CODE=="SET08108") & (AYR_CODE=="2016/7") & (PSL_CODE=="TR1")')
 .groupby('QUE_CODE')[['score']].agg(['sum', 'count']).reset_index())
a['pct'] = round(100*a['score']['sum']/a['score']['count'], 0)
a.columns = ['QUE_CODE', 's', 'n', 'pct']
(a.merge(insque, on='QUE_CODE')
 .merge(inscat, on='CAT_CODE')
 [['QUE_CODE', 'QUE_TEXT', 'CAT_NAME', 'pct']])

,QUE_CODE,QUE_TEXT,CAT_NAME,pct
0,1.1,Staff are good at explaining things.,Learning and Teaching,89.0
1,1.2,Staff made the subject interesting.,Learning and Teaching,82.0
2,1.3,The module was intellectually stimulating.,Learning and Teaching,82.0
3,1.4,The aims and objectives were clearly stated.,Learning and Teaching,89.0
4,1.5,The module was well-organised and ran smoothly.,Learning and Teaching,78.0
5,1.6,The pace was appropriate.,Learning and Teaching,80.0
6,1.7,The level was appropriate.,Learning and Teaching,82.0
7,1.8,The workload was managable.,Learning and Teaching,78.0
8,1.9,I was able to contact module staff when I need...,Learning and Teaching,76.0
9,2.1,The assessment requirements were clearly stated.,Assessment and Feedback,84.0


## 5. Show the frequency chart for module SET08108 for question 4.1

**For each response 1-5 show the number of students who gave that response (Module SET08108, 2016/7, TR1)**

(note that this is not real data, these responses were randomly generated)

In [7]:
(insres
 .query('(MOD_CODE=="SET08108") & (AYR_CODE=="2016/7") & (PSL_CODE=="TR1") & (QUE_CODE=="4.1")')
 [['MOD_CODE', 'RES_VALU', 'SPR_CODE']]
 .groupby(['MOD_CODE', 'RES_VALU'])
 .count().reset_index())

,MOD_CODE,RES_VALU,SPR_CODE
0,SET08108,2,6
1,SET08108,4,10
2,SET08108,5,39
